In [1]:
import datetime
import requests
import json
import numpy as np
import pickle
import os
import folium
import math

## Main function

In [2]:
def dividi(lista, chunk_size):
    
    return [lista[i:i + chunk_size] for i in range(0, len(lista), chunk_size)]

def generate_combinations(chunks):

    combinations = []
    for i in range(len(chunks)):
        for j in range(len(chunks)):
            combinations.append((chunks[i], chunks[j]))
    return combinations

In [3]:
def create_one_matrix(array):
    n = int(np.sqrt(array.shape[0]))  # Calcola n
    # Inizializza una lista per le righe concatenate
    rows = []
    for i in range(n):
        # Concatenazione orizzontale delle matrici lungo una riga
        row = np.concatenate(array[i*n:(i+1)*n], axis=1)
        rows.append(row)
    # Concatenazione verticale delle righe concatenate
    large_matrix = np.concatenate(rows, axis=0)
    return large_matrix

In [24]:
def main(file_path):
    """
    file_path: path al file json con le coordinate dei punti in una città
    Funzione che crea le matrici di distanze e tempi tra tutti i punti della città
    e le salva in due file .npy
    """
    
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    chunks = dividi(data,10)
    
    combinations = generate_combinations(chunks)
    
    formatted_combinations = []
    
    for comb in combinations:
       formatted_partenze = '|'.join([f"{lat},{lon}" for lon, lat in comb[0]]) 
       formatted_arrivi = '|'.join([f"{lat},{lon}" for lon, lat in comb[1]])
       formatted_combinations.append([formatted_partenze, formatted_arrivi])
    
    base_url = 'https://maps.googleapis.com/maps/api/distancematrix/json'
    
    data = []
    
    for i in range(len(formatted_combinations)):

        params = {
            'key':'API_KEY',
            'origins': formatted_combinations[i][0],
            'destinations': formatted_combinations[i][1],
            'departure_time': 1749542400,
            'mode': 'transit'
        }

        r = requests.get(base_url, params=params)


        if r.status_code == 200:
            data.append(json.loads(r.content))

        else:
            print(f"Errore con origine {formatted_data[i][0]} e destinazione {formatted_data[i][1]}: {r.status_code}")


    matrici_distances = []
    matrici_durations = []

    # Itera su ogni elemento di data
    for i in range(len(data)):

        # Inizializza le matrici di distanza e durata con np.nan
        distances = np.full((10, 10), np.nan)  # Usiamo (2, 2) per semplificare l'esempio, dovresti usare (10, 10)
        durations = np.full((10, 10), np.nan)  # Usiamo (2, 2) per semplificare l'esempio, dovresti usare (10, 10)

        # Popola le matrici con i valori appropriati
        for l, row in enumerate(data[i]['rows']):
            for m, element in enumerate(row['elements']):
                if element['status'] == 'OK':
                    distances[l, m] = element['distance']['value']
                    durations[l, m] = element['duration']['value']

        matrici_distances.append(distances)
        matrici_durations.append(durations)
    
    
    array_d = np.array(matrici_distances)
    array_t = np.array(matrici_durations)
    
    matrice_d = create_one_matrix(array_d)
    matrice_t = create_one_matrix(array_t)
    
    
    np.save("../matrices/matrice_distanze_100.npy", matrice_d)
    np.save("../matrices/matrice_durate_100.npy", matrice_t)
    
    print(matrice_d.shape)
    
    return None

In [ ]:
main('../coordinates/coordinates_100.json')

(100, 100)